In [129]:
## Parameters
no_adoption_decline_parameter = 1.0 #  Reasonable probably
full_adoption_decline_parameter = 0.7  # Might be able to better on this. My justification is cited in the writeup for Esteban
sigma = 4.0 
psi   = 0.43 #should estimate from data
local = "Users/jeffreyohl/Downloads/"

# Data 

In [91]:
## Initial API Call
import requests
import pandas as pd
from urllib.parse import quote

def get_county_education_2010():
   api_key = "488b0df48f856b207eae540095aee8cd37926d3a"
   base_url = "https://api.census.gov/data/2010/acs/acs5"
   variables = [
     "B15002_001E",  # Total population 25+
       "B15002_015E",  # Male: Bachelor's degree
       "B15002_016E",  # Male: Master's degree 
       "B15002_017E",  # Male: Professional school degree
       "B15002_018E",  # Male: Doctorate degree
       "B15002_032E",  # Female: Bachelor's degree
       "B15002_033E",  # Female: Master's degree
       "B15002_034E",  # Female: Professional school degree
       "B15002_035E"   # Female: Doctorate degree
    ]
   
   # Build URL
   get_vars = quote(",".join(variables))
   url = f"{base_url}?get=NAME,{get_vars}&for=county:*&in=state:*&key={api_key}"
   
   print(f"Requesting URL: {url}")
   
   try:
       response = requests.get(url)
       print(f"Status code: {response.status_code}")
       
       if response.status_code != 200:
           raise Exception(f"API returned status code {response.status_code}")
           
       # Parse JSON into DataFrame
       data = response.json()
       df = pd.DataFrame(data[1:], columns=data[0])
       
       # Convert to numeric
       for var in variables:
           df[var] = pd.to_numeric(df[var])
           
       # Calculate bachelors or higher (sum male and female)
       df['bachelors_plus'] = (
           df['B15002_015E'] + df['B15002_016E'] + df['B15002_017E'] + df['B15002_018E'] +  # Male
           df['B15002_032E'] + df['B15002_033E'] + df['B15002_034E'] + df['B15002_035E']    # Female
       )
       
       # Calculate percentage
       df['pct_bachelors_plus'] = (df['bachelors_plus'] / df['B15002_001E']).round(8)
       
       # Select and rename columns
       final_df = df[['NAME', 'state', 'county', 'pct_bachelors_plus']]
       
       # Save to CSV
       #final_df.to_csv('county_bachelors_2006_2010.csv', index=False)
       #print("\nSaved to county_bachelors_2006_2010.csv")
       
       return final_df
       
   except requests.exceptions.RequestException as e:
       print(f"Request failed: {e}")
       raise
   except ValueError as e:
       print(f"JSON parsing failed: {e}")
       print(f"Raw response: {response.text}")
       raise

if __name__ == '__main__':
   try:
       county_data = get_county_education_2010()
       print("\nFirst few rows:")
       print(county_data.head())
   except Exception as e:
       print(f"Error: {e}")

Requesting URL: https://api.census.gov/data/2010/acs/acs5?get=NAME,B15002_001E%2CB15002_015E%2CB15002_016E%2CB15002_017E%2CB15002_018E%2CB15002_032E%2CB15002_033E%2CB15002_034E%2CB15002_035E&for=county:*&in=state:*&key=488b0df48f856b207eae540095aee8cd37926d3a
Status code: 200

First few rows:
                                NAME state county  pct_bachelors_plus
0  Las Marías Municipio, Puerto Rico    72    083            0.123230
1  San Germán Municipio, Puerto Rico    72    125            0.196672
2     Comerío Municipio, Puerto Rico    72    045            0.119509
3   Canóvanas Municipio, Puerto Rico    72    029            0.185542
4      Rincón Municipio, Puerto Rico    72    117            0.188367


# Rest of raw data

In [ ]:

import pandas as pd 

import pandas as pd 
# Raw Data.
in_distance = local + "bilateral_distances.dta"
distances = pd.read_stata(in_distance") # note we can't upload this too github it is too big
flows = pd.read_stata("../data/Monte et al 2018 Replication Folder/CMLEE.dta") # i am pulling from esteban's dataset for this
land_area = pd.read_csv('../data/Raw Data/CountyCentroids2010.txt',  # From Census. 
                delimiter='\t',
                encoding='latin-1')
land_area = (
    land_area[['GEOID', 'ALAND']]
    .drop_duplicates()                # remove duplicates based on entire row
    .sort_values(by='GEOID', ascending=True)
)

land_area = land_area[['ALAND', 'GEOID']]



/var/folders/g9/ggrmf_5j6tx4rv5qdhs0slq40000gn/T/ipykernel_52815/1431070172.py:6: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  flows = pd.read_stata("../data/Monte et al 2018 Replication Folder/CMLEE.dta") # i am pulling from esteban's dataset for this


In [93]:

### First step is to find the eligible counties for our analysis.

# Use this for calculating data that we have commute flows, etc. on. 
pop_weights = (
    flows[['state_county_work', 'workplace_emp']]
    .drop_duplicates()                # remove duplicates based on entire row
    .sort_values(by='state_county_work', ascending=True)
)


# Construct FIPs without leading zeros for consistency with Monte et al code
county_data['FIPS'] = county_data['state'].astype(str) + county_data['county'].astype(str)
county_data['FIPS'] = county_data['FIPS'].str.lstrip('0')
county_data['FIPS']  = county_data['FIPS'].astype(int)
county_data.sort_values(by="FIPS",ascending=True)


# --- Merge Education Data and Population data  -- 
df_merged = county_data.merge(
    pop_weights, 
    left_on='FIPS', 
    right_on='state_county_work', 
    how='inner'
).sort_values('FIPS')


counties_for_inclusion = df_merged.FIPS.unique() # some dont have education data, we can't include those in our analysis

#####

# Only counties in all datasets can be used for analysis, filter down to those (we drop about 1% of counties, not bad)

In [94]:


distances = distances[
    distances['state_county_work'].isin(counties_for_inclusion) &
    distances['state_county_res'].isin(counties_for_inclusion)
]

In [95]:
flows = flows[
    flows['state_county_work'].isin(counties_for_inclusion) &
    flows['state_county_res'].isin(counties_for_inclusion)
]

In [96]:
land_area = land_area[
    land_area['GEOID'].isin(counties_for_inclusion) 
]

# Compute college-shares for later.

In [97]:
# Extract the relevant arrays
pct = df_merged['pct_bachelors_plus'].values
emp = df_merged['workplace_emp'].values
# 2) Extract arrays for shares (pct) and employment (emp).
pct = df_merged['pct_bachelors_plus'].to_numpy()
emp = df_merged['workplace_emp'].to_numpy()

# 3) Vectorized pairwise workplace-weighted averages:
#    arr[i,j] = (pct[i]*emp[i] + pct[j]*emp[j]) / (emp[i] + emp[j])
numerator   = (pct * emp)[:, None] + (pct * emp)[None, :]
denominator = emp[:, None] + emp[None, :]
scaling = numerator / denominator ## this is the pairwise workplace weighted average of the shares of bachelors degrees, we will use this to scale commute costs.
####
scaling.shape


(3083, 3083)

# B_ni_hat - this is the counterfactual decline we will apply in our counterfactual

In [98]:
import numpy as np


# B_ni_hat = 1.0 - 0.3 * scaling
B_ni_hat = (no_adoption_decline_parameter 
            - (no_adoption_decline_parameter - full_adoption_decline_parameter)*scaling)

# Because scaling is symmetric, we really only need the lower triangle:
B_ni_hat_lower = np.tril(B_ni_hat)

# Now mirror to get a fully symmetric NxN:
B_ni_hat = B_ni_hat_lower + B_ni_hat_lower.T - np.diag(np.diag(B_ni_hat_lower))


In [128]:
df_B_ni_hat = pd.DataFrame(B_ni_hat, index=df_merged.FIPS, columns=df_merged.FIPS)
output_Bni_hat = local + "B_ni_hat.csv"
df_B_ni_hat.to_csv(output_Bni_hat)

NameError: name 'local' is not defined

# Convert things to np arrays

In [99]:


# 1. Pivot to make each row a unique "res" and each column a unique "work"
pivoted = distances.pivot(
    index='state_county_res',
    columns='state_county_work',
    values='dist'
)

# 2. Sort the index and columns so that the smallest FIPS codes appear first
pivoted = pivoted.sort_index(axis=0).sort_index(axis=1)

# 3. Convert the pivoted DataFrame to a NumPy array
dist_ni = pivoted.to_numpy()

# I am keeping everything in ascending by FIPs order

In [111]:
## Pull key quantities from Esteban's data. 
residence_emp = (
    flows[['state_county_res', 'residence_emp']]
    .drop_duplicates()                # remove duplicates based on entire row
    .sort_values(by='state_county_res', ascending=True)
)


workplace_emp = (
    flows[['state_county_work', 'workplace_emp']]
    .drop_duplicates()                # remove duplicates based on entire row
    .sort_values(by='state_county_work', ascending=True)
)


avgw_pow = (
    flows[['state_county_work', 'avgw_pow']]
    .drop_duplicates()  # remove duplicate rows
    .sort_values(by='state_county_work')
)
land_area = land_area.sort_values(by='GEOID', ascending=True)




In [112]:
# Vectors we need for the model
workemp   =  np.array(workplace_emp['workplace_emp'])
workwage  = np.array(avgw_pow['avgw_pow'])
    

H_n = np.array(land_area['ALAND'])
    
# Suppose R (residents), D, vbar are also known (random example):
residence_emp    = np.array(residence_emp['residence_emp']) 

In [113]:
len(workemp)

3083

In [114]:
# 1) Compute the pivot table using sum for duplicates (if needed) and fill with 0
df_pivot = flows.pivot_table(
    index='state_county_res',
    columns='state_county_work',
    values='commuting',
    aggfunc='sum',      # sum over duplicates
    fill_value=0
)

# 2) Sort rows and columns in ascending order
df_pivot = df_pivot.sort_index().sort_index(axis=1)

# 3) Divide by the sum of commuting over the entire DataFrame, so each cell becomes a share
total_commuting = flows['commuting'].sum()
df_pivot = df_pivot / total_commuting


lambda_ni = np.array(df_pivot)
lambda_nR = lambda_ni.sum(axis=1)
# Broadcast division over each row using [:, None] to match dimensions
lambda_ni_given_n = lambda_ni / lambda_nR[:, None]   # shape: (N, M)
vbar_n = (lambda_ni_given_n * workwage).sum(axis=1)  # shape: (N,)

# Solve for productivitites

In [115]:
import numpy as np

def rhsProductivities(A, L, w, d, R, D, sigma, vbar):
    """
    Returns an array 'labIncomeHat' of shape (N,), where:
      labIncomeHat[i] = sum_n [
        L[i]*(d[n,i]*w[i]/A[i])^(1-sigma)
        / sum_k [ L[k]*(d[n,k]*w[k]/A[k])^(1-sigma) ]
      ] * [vbar[n]*R[n] + D[n]].

    Parameters
    ----------
    A : array of shape (N,)
        Current guess of A^(sigma-1) or A (depending on usage).
        Make sure A > 0.
    L, w : arrays of shape (N,)
        Employment and wages for each region i.
    d    : (N x N) array with d[n,i] = distance factor from n to i.
    R, D, vbar : arrays of shape (N,)
        R[n], D[n], vbar[n] for region n.
    sigma : float
        Elasticity (> 1).
    
    Returns
    -------
    labIncomeHat : array of shape (N,)
        Model-implied labor income for each i, i.e. the sum over n
        of fraction_{n,i} * (vbar[n]*R[n] + D[n]).
    """
    N = len(A)
    # M[n,i] = L[i]*(d[n,i]*w[i]/A[i])^(1-sigma)
    M = L * np.power(d * (w / A), 1.0 - sigma)  # shape (N, N), numerator
    
    # denom[n] = sum_k M[n,k]
    denom = np.sum(M, axis=1)                   # shape (N,)
    
    # X[n] = vbar[n]*R[n] + D[n]
    X = vbar * R + D                            # shape (N,)
    
    # fraction_{n,i} = M[n,i] / denom[n].
    # partial_term[i,n] = fraction_{n,i} * X[n] => we use transpose.
    M_T = M.T  # shape (N, N), M_T[i,n] = M[n,i]
    summand = (M_T / denom) * X  # broadcast X[n], denom[n]
    labIncomeHat = np.sum(summand, axis=1) # sum over n => shape (N,)
    
    return labIncomeHat


def productivities(
    workemp,      # L_i
    workwage,     # w_i
    resExp,       # dictionary or struct with R, D, vbar
    distmatrix,   # NxN
    sigma,        # elasticity
    psi,          # distance exponent
    detailsYN=True,
    max_iter=1000,
    tol=1e-6
):
    """
    Python version of the Mathematica-style tâtonnement for solving
      w_i L_i == sum_n fraction_{n,i} * [vbar[n]*R[n] + D[n]],
    where fraction_{n,i} = M[n,i]/sum_k M[n,k], M[n,i]=(L_i*(d_{n,i}*w_i/A_i)^(1-sigma)).
    
    We iterate on A^(sigma-1) until the implied w_i L_i matches the actual w_i L_i.
    
    Returns
    -------
    A_final : array of shape (N,) = A_i^(1/(sigma-1)), i.e. the final productivities.
    """

    ncounties = len(workemp)
    
    #------------------------------------------------
    # 1) Preprocess / scaling as per your code
    #    You can adjust these scalings to match your
    #    original Mathematica approach exactly.
    #------------------------------------------------
    
    
    L = workemp  # employees in hundreds
    w = workwage   #wage in tens of thousands thousands

    # Let d = distmatrix^(psi)
    d = distmatrix**psi

    # Unpack R, D, vbar from resExp dict (adjust as needed)
    R = resExp["R"]       # array of shape (N,)
    D_ = resExp["D"]      # array of shape (N,)
    vbar = resExp["vbar"] # array of shape (N,)

    # We'll compute actual "labIncome" = w_i L_i, but note
    # we used w=workwage/1e4 => actual w_i L_i is
    #    (workwage[i]/1e4)*workemp[i] ...
    # If your code wants it in "millions" => divide by 1e6
    # or do whichever scaling you used in Mathematica:
    labIncome = (L * w)  # in
    
    # Step size (partial adjustment)
    lam = 0.990
    
    #------------------------------------------------
    # 2) Initialize A0 as A^(sigma-1) = 1
    #------------------------------------------------
    A0 = np.ones(ncounties)
    
    #------------------------------------------------
    # 3) Iteration
    #------------------------------------------------
    error = 1.0  # track max gap
    c = 0
    
    while error > tol and c < max_iter:
        c += 1
        
        # Model-implied income under current guess A0
        # Note we pass D_ (because "D" is a python builtin),
        # but just rename it for clarity.
        labIncomeHat = rhsProductivities(A0, L, w, d, R, D_, sigma, vbar)
        print("lab income")
        print(labIncome)
        print("total labor income true")
        print(np.sum(labIncome))
        print("lab income hat")
        print(labIncomeHat)
        print("total lab income hat")
        print(np.sum(labIncomeHat))
        # Compare to actual labIncome => gap[i] = actual[i]/hat[i]
        gap = labIncome / labIncomeHat
       
        # "dist" is the maximum relative difference from 1
        error = np.max(np.abs(gap - 1.0))
        distmin = np.min(np.abs(gap - 1.0))
        
        # Partial update: A1 = [ lam + (1-lam)*gap ] * A0
        A1 = (lam + (1.0 - lam)*gap) * A0
        
        # Rescale so that mean(A1) = 1 (like your code)
        A1_mean = np.mean(A1) 
        A0 = A1 / A1_mean # normalize to mean 1.
        print("mean A0")
        print(np.mean(A0))
        print("A0")
        print(A0)
        # Optional: print diagnostics
        if detailsYN and c % 50 == 1 and error > 0.01:
            print(f"Iteration {c}, max gap={error:8.5f}, min gap={distmin:8.5f}, "
                  f"A0 in [{A0.min():.3e}, {A0.max():.3e}]")
            

    if detailsYN:
        print(f"End of iteration {c}, max gap={error:8.5f}, "
              f"A0 in [{A0.min():.3e}, {A0.max():.3e}]")
    
    #------------------------------------------------
    # 4) Exponentiate back to get true A
    #    A^(1/(sigma-1)) and rescale so mean(...)=1
    #------------------------------------------------
    exponent = 1.0 / (sigma - 1.0)
    A_linear = A0**exponent
    A_linear /= np.mean(A_linear)
    
    return A_linear


# ------------------------------------------------------------------------
# EXAMPLE USAGE (dummy example)
if __name__ == "__main__":
    np.random.seed(0)
    ncounties = len(workemp)
    


    total_vbar_R = np.sum(vbar_n * residence_emp)  # scalar
    total_wage_emp = np.sum(workwage * workemp)  # scalar

    difference = total_vbar_R - total_wage_emp
    # allocate deficits equally. 
    D     = np.zeros(ncounties) - (difference)/ ncounties     # no deficits -- need from Yulia 
    # Put them in resExp
    resExp = {"R": residence_emp, "D": D, "vbar": vbar_n}
    
   
    
    A_solution = productivities(
        workemp=workemp,
        workwage=workwage,
        resExp=resExp,
        distmatrix=dist_ni,
        sigma=sigma,
        psi=psi,
        detailsYN=True,
        max_iter=1000,
        tol=1e-8
    )
    print("\nFinal productivities A:", A_solution)


lab income
[6.40599025e+08 2.90766793e+09 4.67938012e+08 ... 5.81472986e+08
 1.99311006e+08 1.54419001e+08]
total labor income true
8756900399015.042
lab income hat
[1.34130140e+09 7.35428643e+09 1.00756635e+09 ... 3.20226942e+08
 2.12988406e+08 3.14725137e+08]
total lab income hat
8756900698862.605
mean A0
1.0000000000000002
A0
[0.99642721 0.9956036  0.99629528 ... 1.00983163 1.0010167  0.99655795]
Iteration 1, max gap=197.51064, min gap= 0.00017, A0 in [9.917e-01, 2.980e+00]
lab income
[6.40599025e+08 2.90766793e+09 4.67938012e+08 ... 5.81472986e+08
 1.99311006e+08 1.54419001e+08]
total labor income true
8756900399015.042
lab income hat
[1.32350606e+09 7.23777209e+09 9.93718655e+08 ... 3.28064831e+08
 2.12551855e+08 3.09984288e+08]
total lab income hat
8756900698862.603
mean A0
1.0
A0
[0.99369239 0.99204985 0.99342978 ... 1.02010244 1.0028218  0.99396396]
lab income
[6.40599025e+08 2.90766793e+09 4.67938012e+08 ... 5.81472986e+08
 1.99311006e+08 1.54419001e+08]
total labor income tru

# Save productivities

In [116]:
import numpy as np
output_folder_name = "../output/"
prod_path_name = output_folder_name + "productivities.csv"

np.savetxt(prod_path_name, A_solution, delimiter=',')


# Estimating Equation for $\phi$

In [122]:
import numpy as np
import pandas as pd
from linearmodels.iv import IV2SLS
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
# Suppose you already have:
#   lambda_ni: 2D array of shape (N, I)
#   workwage:  1D array of shape (I,) for w_i
#   distm:     2D array of shape (N, I) for dist_{ni}
#   A_solution:1D array of shape (I,) (the excluded instrument)
#   N = number of residence areas, I = number of workplaces

# 1) Build a long-format DataFrame, dropping rows where lambda_ni == 0
rows = []
N, I = lambda_ni.shape
for n in range(N):
   
    for i in range(I):
        if lambda_ni[n, i] > 0:
            rows.append({
                'n': n,  # residence index
                'i': i,  # workplace index
                'log_lambda': np.log(lambda_ni[n, i]),
                'log_wage':   np.log(workwage[i]),
                'log_dist':   np.log(dist_ni[n, i]),
                'log_A_inst':     np.log(A_solution[i])
            })
df = pd.DataFrame(rows)
def demean_two_way(df, y_col, x_cols, id1_col, id2_col, max_iter=100, tol=1e-8):
    """
    Demean variables for two-way fixed effects using iterative method.
    """
    y = df[y_col].copy()
    X = df[x_cols].copy()
    
    # Initialize
    y_prev = y.copy() + 1
    iter_count = 0
    
    while (np.abs(y - y_prev) > tol).any() and iter_count < max_iter:
        y_prev = y.copy()
        
        # Demean with respect to id1
        y = y - y.groupby(df[id1_col]).transform('mean')
        X = X - X.groupby(df[id1_col]).transform('mean')
        
        # Demean with respect to id2
        y = y - y.groupby(df[id2_col]).transform('mean')
        X = X - X.groupby(df[id2_col]).transform('mean')
        
        iter_count += 1
    
    return y, X
y_demean, X_demean = demean_two_way(
    df,
    y_col='log_lambda',
    x_cols=['log_dist'],
    id1_col='n',
    id2_col='i'
)

# Run regression on demeaned variables
X_demean = sm.add_constant(X_demean)  # Add constant
model = sm.OLS(y_demean, X_demean)
results = model.fit()



In [123]:
phi = -results.params['log_dist']
phi

np.float64(3.9797599518994926)

# Estimating Equation for $\epsilon$

In [124]:
df['y'] = df['log_lambda'] + 3.9783*df['log_dist']

# 3) Sweep out fixed effects by demeaning over n. (not sure how sketchy this is)
#    For each residence n, subtract the mean (over i) from every variable.
df['y_demeaned'] = df['y'] - df.groupby('n')['y'].transform('mean')
df['log_wage_demeaned'] = df['log_wage'] - df.groupby('n')['log_wage'].transform('mean')
df['log_A_inst_demeaned'] = df['log_A_inst'] - df.groupby('n')['log_A_inst'].transform('mean')

# 4) Run the 2SLS regression (no constant, since we've already demeaned).
#    y_demeaned = epsilon * log_wage_demeaned + error
#    with log_wage_demeaned instrumented by A_inst_demeaned.
formula = 'y_demeaned ~ 0 + [log_wage_demeaned ~ log_A_inst_demeaned]'
mod = IV2SLS.from_formula(formula, data=df)
res = mod.fit(cov_type='robust')  # Or choose your preferred cov_type
res

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:             y_demeaned   R-squared:                      0.2232
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2232
No. Observations:               44047   F-statistic:                    8807.0
Date:                Fri, Jan 31 2025   P-value (F-stat)                0.0000
Time:                        14:24:03   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                 
=====================================================================================
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
log_wage_demeaned     3.1358     0.0334     93.846     0.0000      3.0703      3.2013
=====================================================================================

Endogenous: log_wage_demeaned
Instruments: log_A_inst_demeaned
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x16b015590

In [125]:
epsilon = res.params['log_wage_demeaned']
epsilon

np.float64(3.135795041441956)

In [126]:
import pandas as pd

# Create a DataFrame with the parameters
parameters = pd.DataFrame({
    'Parameter': ['epsilon', 'phi'],
    'Value': [epsilon, phi]
})

# Define the output file path
output_file_path = output_folder_name + "commuting_parameters.csv"

# Save the DataFrame to a CSV file
parameters.to_csv(output_file_path, index=False)